In [22]:
# import package
# installed via pip
from emtracks.particle import trajectory_solver # main solver object
from emtracks.conversions import one_gev_c2_to_kg # conversion for q factor (transverse momentum estimate)
from emtracks.tools import *#InitConds # initial conditions namedtuple
from emtracks.mapinterp import get_df_interp_func  # factory function for creating Mu2e DS interpolation function
from emtracks.Bdist import get_B_df_distorted
from emtracks.interpolations import *
import matplotlib.animation as animation
import numpy as np
from scipy.constants import c, elementary_charge
import pandas as pd
import pickle as pkl
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import math
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = [24,16] # bigger figures
from matplotlib import style
style.use('fivethirtyeight')
import os

from joblib import Parallel, delayed
import multiprocessing
from tqdm.notebook import tqdm

testdir = "/home/darren/Desktop/plots/"
datadir = "/home/shared_data/"
plotdir = datadir+"plots/randomphi/"
mapdir = datadir+"Bmaps/"
date = "/7-9/"
newdir = datadir+'testmao/'

maomapdir = '/home/shared_data/Bmaps/Mau10/combined/'

In [23]:
def get_uniform_phi(N):
    phis = np.linspace(0, 2*math.pi, N)
    return phis

def get_uniform_theta(N):
    thetas = np.linspace(0, math.pi, N)
    return thetas

files = sorted(os.listdir(maomapdir)) [:11]

In [24]:
def get_funcs(files): 
    #nominal field
    df_Mu2e = pd.read_pickle(mapdir+"Mu2e_DSMap_V13.p")
    B_Mu2e_func = get_df_interp_func(mapdir+"Mu2e_DSMap_V13.p", gauss=False)
    data = {'nom':B_Mu2e_func}
    #new field
    for file in files:
        param = file.split('x')[1].split('_')[1]
        with open(maomapdir+'/'+file, 'rb') as pickle_file:
            data.update({param:get_df_interp_func(df = pkl.load(pickle_file), gauss=False)})
    return data

In [25]:
def run_solver(phi, theta, field, fielddescrip, N_calc, xnaught, ynaught, znaught, name):
    ic_Mu2e = InitConds(t0=0., tf=4e-8, N_t=N_calc,
                    x0=xnaught, y0=ynaught, z0=znaught,
                    p0=104.96, theta0=theta, phi0=phi)
    e_solver = trajectory_solver(ic_Mu2e, B_func=field, bounds=bounds_Mu2e)
    sol = e_solver.solve_trajectory(verbose = False, atol=1e-10, rtol=1e-10)
    e_solver.dataframe['r'] = ((e_solver.dataframe['x'])**2 + (e_solver.dataframe['y'])**2)**(1/2)
    e_solver.dataframe['rfromcenter'] = ((e_solver.dataframe['x'])**2 + (e_solver.dataframe['y'])**2)**(1/2)
    filename = f'{fielddescrip}_{ic_Mu2e.theta0:0.3f}_{ic_Mu2e.phi0:0.3f}_{ic_Mu2e.x0:0.3f}_{name}.pkl'
    e_solver.to_pickle(newdir+filename)   

In [26]:
def runbothsolvers(phi, theta, fielddata, N_calc, xnaught, ynaught, znaught):
    keys = list(fielddata.keys())
    values = list(fielddata.values())
    
    run_solver(phi, theta, values[0], str(keys[0]), N_calc, xnaught, ynaught, znaught, 'nom')
        
    for i in range(1, len(keys), 1):
        run_solver(phi, theta, values[i], str(keys[i]), N_calc, xnaught, ynaught, znaught, 'dis')

In [27]:
def savepickle(N, files, ncalc = 20001):
    phis = get_uniform_phi(N)
    thetas = get_uniform_theta(N)
    data = get_funcs(files)
    
    numcpu = multiprocessing.cpu_count()
    Parallel(n_jobs = numcpu)(delayed(runbothsolvers)(phi, theta, data, ncalc, 0.054094482, 0.03873037, 5.988900879) for theta in tqdm(thetas[:3], desc = 'theta') for phi in phis[:3
    ])


In [28]:
savepickle(3, files)

In [8]:
cpu_count = multiprocessing.cpu_count()

In [14]:
files = sorted(os.listdir(newdir))
e_solvernom = trajectory_solver.from_pickle(newdir+files[0])
f = interp_cole(e_solvernom.dataframe, 13)

In [17]:
len(files)

108

In [19]:
for file in files:
    name = file.split('_')[0]
    print(name)

0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.10
0.10
0.10
0.10
0.10
0.10
0.10
0.10
0.10
0.20
0.20
0.20
0.20
0.20
0.20
0.20
0.20
0.20
0.30
0.30
0.30
0.30
0.30
0.30
0.30
0.30
0.30
0.40
0.40
0.40
0.40
0.40
0.40
0.40
0.40
0.40
0.50
0.50
0.50
0.50
0.50
0.50
0.50
0.50
0.50
0.60
0.60
0.60
0.60
0.60
0.60
0.60
0.60
0.60
0.70
0.70
0.70
0.70
0.70
0.70
0.70
0.70
0.70
0.80
0.80
0.80
0.80
0.80
0.80
0.80
0.80
0.80
0.90
0.90
0.90
0.90
0.90
0.90
0.90
0.90
0.90
1.00
1.00
1.00
1.00
1.00
1.00
1.00
1.00
1.00
nom
nom
nom
nom
nom
nom
nom
nom
nom
